## 1. Mount Google Drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Setup Environment

In [9]:
# Chuyển đến thư mục project
%cd "/content/drive/MyDrive/Đồ án Python/coffee_project"

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', module='sklearn')

# Import libraries
import sys
import os
sys.path.append(os.getcwd())

from src.models.trainer import ModelTrainer, TrainingConfig
from src.models.evaluator import ClusteringEvaluator
from src.models.tuning import HyperparameterTuner, TuningConfig
import pandas as pd
import logging

# Setup logger
logging.basicConfig(level=logging.INFO, format='%(message)s')

print("✅ Environment setup complete!")

/content/drive/MyDrive/Đồ án Python/coffee_project
✅ Environment setup complete!


## 3. Hyperparameter Tuning (Advanced)
Grid search tất cả models để tìm best hyperparameters

In [14]:
print("="*80)
print("K-MEANS HYPERPARAMETER TUNING")
print("="*80)

# 1. Thiết lập Grid cho K-Means
# n_clusters: Số lượng cụm muốn thử (quan trọng nhất)
# init: Phương pháp khởi tạo tâm cụm
# n_init: Số lần chạy thuật toán với các tâm ngẫu nhiên khác nhau
kmeans_grid = {
    "n_clusters": [3, 4, 5, 6, 7, 8, 9, 10],
    "init": ["k-means++"],
    "n_init": [10, 20],
    "max_iter": [300]
}

print("Grid parameters:")
for key, values in kmeans_grid.items():
    print(f"  {key}: {values}")

total = 1
for v in kmeans_grid.values():
    total *= len(v)
print(f"\nTotal combinations: {total}")

# 2. Khởi tạo Tuner
evaluator = ClusteringEvaluator()
tuning_config = TuningConfig(
    data_path="data/processed/encoded_data.csv",
    results_path="results/kmeans_tuning.csv",  # Đổi tên file lưu
    metric_selection="silhouette"              # Metric chính để chọn model
)
tuner = HyperparameterTuner(config=tuning_config, evaluator=evaluator)

# 3. Chạy Tuning
# Lưu ý: "kmeans" phải khớp với tên model trong ModelTrainer
tuner.run_grid_search("kmeans", kmeans_grid)

# 4. Lưu kết quả tổng hợp
tuner.save_results()

# 5. Lấy kết quả tốt nhất và Train lại
df_results = tuner.get_summary()

if len(df_results) > 0:
    best = df_results.iloc[0]

    print("\n" + "="*80)
    print("🏆 BEST K-MEANS CONFIG")
    print("="*80)
    # Hiển thị các tham số của K-Means
    print(f"  n_clusters             : {int(best['n_clusters'])}")
    print(f"  init                   : {best['init']}")
    print(f"  n_init                 : {best['n_init']}")

    print(f"\n  Silhouette Score       : {best['silhouette']:.4f}")
    print(f"  Calinski-Harabasz      : {best['calinski_harabasz']:.2f}")
    print(f"  Davies-Bouldin Index   : {best['davies_bouldin']:.4f}")

    # Train và lưu model tốt nhất
    print("\n🔄 Training best model...")

    # Chuẩn bị params (n_clusters thường được tách riêng trong logic của trainer)
    best_params = {
        "init": best['init'],
        "n_init": int(best['n_init']),
        "max_iter": int(best['max_iter'])
    }

    config = TrainingConfig(
        data_path="data/processed/encoded_data.csv",
        model_type="kmeans",
        n_clusters=int(best['n_clusters']), # Quan trọng: KMeans cần n_clusters
        model_params=best_params,
        model_path="results/kmeans_best.pkl"
    )

    trainer = ModelTrainer(config=config, evaluator=evaluator)
    trainer.load_data()
    trainer.train_model()
    trainer.save_model()
    trainer.save_labels("results/kmeans_best_labels.csv")

    print("✅ Best K-Means model saved!")
    print("   Model: results/kmeans_best.pkl")
    print("   Labels: results/kmeans_best_labels.csv")

    # Show top 5 configs
    print("\n" + "="*80)
    print("📊 TOP 5 CONFIGS")
    print("="*80)
    # Chỉ chọn các cột có ý nghĩa với K-Means
    cols = ['n_clusters', 'init', 'n_init',
            'silhouette', 'calinski_harabasz', 'davies_bouldin']
    # Lọc các cột thực sự tồn tại trong df kết quả (đề phòng lỗi key)
    cols = [c for c in cols if c in df_results.columns]
    print(df_results[cols].head(5).to_string(index=False))

else:
    print("⚠️ No valid results found")

📂 Loading data for tuning from data/processed/encoded_data.csv...
  ✓ Data loaded: (3685, 58)

🚀 Starting Grid Search for: KMEANS
  Total combinations to test: 16


K-MEANS HYPERPARAMETER TUNING
Grid parameters:
  n_clusters: [3, 4, 5, 6, 7, 8, 9, 10]
  init: ['k-means++']
  n_init: [10, 20]
  max_iter: [300]

Total combinations: 16


  [1/16] KMEANS_n_clusters3_initk-means++_n_init10_max_iter300: Sil=0.2544, CH=1269.22, DB=1.5338, Clusters=3
  [2/16] KMEANS_n_clusters3_initk-means++_n_init20_max_iter300: Sil=0.2544, CH=1269.22, DB=1.5338, Clusters=3
  [3/16] KMEANS_n_clusters4_initk-means++_n_init10_max_iter300: Sil=0.2355, CH=1030.48, DB=1.8631, Clusters=4
  [4/16] KMEANS_n_clusters4_initk-means++_n_init20_max_iter300: Sil=0.2355, CH=1030.48, DB=1.8631, Clusters=4
  [5/16] KMEANS_n_clusters5_initk-means++_n_init10_max_iter300: Sil=0.2352, CH=894.63, DB=1.7419, Clusters=5
  [6/16] KMEANS_n_clusters5_initk-means++_n_init20_max_iter300: Sil=0.2352, CH=894.63, DB=1.7419, Clusters=5
  [7/16] KMEANS_n_clusters6_initk-means++_n_init10_max_iter300: Sil=0.1937, CH=820.04, DB=1.8492, Clusters=6
  [8/16] KMEANS_n_clusters6_initk-means++_n_init20_max_iter300: Sil=0.1938, CH=820.06, DB=1.8499, Clusters=6
  [9/16] KMEANS_n_clusters7_initk-means++_n_init10_max_iter300: Sil=0.2019, CH=777.00, DB=1.6383, Clusters=7
  [10/16] KMEAN

model_type  n_clusters  silhouette  calinski_harabasz  davies_bouldin
    kmeans           3    0.254433        1269.217570        1.533831
    kmeans           3    0.254433        1269.217570        1.533831
    kmeans           4    0.235535        1030.477913        1.863101

🏆 BEST K-MEANS CONFIG
  n_clusters             : 3
  init                   : k-means++
  n_init                 : 10

  Silhouette Score       : 0.2544
  Calinski-Harabasz      : 1269.22
  Davies-Bouldin Index   : 1.5338

🔄 Training best model...


💾 Labels saved: results/kmeans_best_labels.csv


✅ Best K-Means model saved!
   Model: results/kmeans_best.pkl
   Labels: results/kmeans_best_labels.csv

📊 TOP 5 CONFIGS
 n_clusters      init  n_init  silhouette  calinski_harabasz  davies_bouldin
          3 k-means++      10    0.254433        1269.217570        1.533831
          3 k-means++      20    0.254433        1269.217570        1.533831
          4 k-means++      10    0.235535        1030.477913        1.863101
          4 k-means++      20    0.235535        1030.477913        1.863101
          5 k-means++      10    0.235202         894.634671        1.741909


In [15]:
print("="*80)
print("K-MEANS HYPERPARAMETER TUNING")
print("="*80)

# 1. Thiết lập Grid cho K-Means
# n_clusters: Số lượng cụm muốn thử (quan trọng nhất)
# init: Phương pháp khởi tạo tâm cụm
# n_init: Số lần chạy thuật toán với các tâm ngẫu nhiên khác nhau
kmeans_grid = {
    "n_clusters": [3, 4, 5, 6, 7, 8, 9, 10],
    "init": ["k-means++"],
    "n_init": [10, 20],
    "max_iter": [300]
}

print("Grid parameters:")
for key, values in kmeans_grid.items():
    print(f"  {key}: {values}")

total = 1
for v in kmeans_grid.values():
    total *= len(v)
print(f"\nTotal combinations: {total}")

# 2. Khởi tạo Tuner
evaluator = ClusteringEvaluator()
tuning_config = TuningConfig(
    data_path="data/processed/encoded_data.csv",
    results_path="results/kmeans_composite_tuning.csv",
    metric_selection="composite",
    silhouette_weight=0.4,
    calinski_weight=0.3,
    davies_weight=0.3
)
tuner = HyperparameterTuner(config=tuning_config, evaluator=evaluator)

# 3. Chạy Tuning
# Lưu ý: "kmeans" phải khớp với tên model trong ModelTrainer
tuner.run_grid_search("kmeans", kmeans_grid)

# 4. Lưu kết quả tổng hợp
tuner.save_results()

# 5. Lấy kết quả tốt nhất và Train lại
df_results = tuner.get_summary()

if len(df_results) > 0:
    best = df_results.iloc[0]

    print("\n" + "="*80)
    print("🏆 BEST K-MEANS CONFIG")
    print("="*80)
    # Hiển thị các tham số của K-Means
    print(f"  n_clusters             : {int(best['n_clusters'])}")
    print(f"  init                   : {best['init']}")
    print(f"  n_init                 : {best['n_init']}")

    print(f"\n  Silhouette Score       : {best['silhouette']:.4f}")
    print(f"  Calinski-Harabasz      : {best['calinski_harabasz']:.2f}")
    print(f"  Davies-Bouldin Index   : {best['davies_bouldin']:.4f}")

    # Train và lưu model tốt nhất
    print("\n🔄 Training best model...")

    # Chuẩn bị params (n_clusters thường được tách riêng trong logic của trainer)
    best_params = {
        "init": best['init'],
        "n_init": int(best['n_init']),
        "max_iter": int(best['max_iter'])
    }

    config = TrainingConfig(
        data_path="data/processed/encoded_data.csv",
        model_type="kmeans",
        n_clusters=int(best['n_clusters']), # Quan trọng: KMeans cần n_clusters
        model_params=best_params,
        model_path="results/kmeans_composite_best.pkl"
    )

    trainer = ModelTrainer(config=config, evaluator=evaluator)
    trainer.load_data()
    trainer.train_model()
    trainer.save_model()
    trainer.save_labels("results/kmeans_best_composite_labels.csv")

    print("✅ Best K-Means model saved!")
    print("   Model: results/kmeans_composite_best.pkl")
    print("   Labels: results/kmeans_best_composite_labels.csv")

    # Show top 5 configs
    print("\n" + "="*80)
    print("📊 TOP 5 CONFIGS")
    print("="*80)
    # Chỉ chọn các cột có ý nghĩa với K-Means
    cols = ['n_clusters', 'init', 'n_init',
            'silhouette', 'calinski_harabasz', 'davies_bouldin']
    # Lọc các cột thực sự tồn tại trong df kết quả (đề phòng lỗi key)
    cols = [c for c in cols if c in df_results.columns]
    print(df_results[cols].head(5).to_string(index=False))

else:
    print("⚠️ No valid results found")

📂 Loading data for tuning from data/processed/encoded_data.csv...
  ✓ Data loaded: (3685, 58)

🚀 Starting Grid Search for: KMEANS
  Total combinations to test: 16


K-MEANS HYPERPARAMETER TUNING
Grid parameters:
  n_clusters: [3, 4, 5, 6, 7, 8, 9, 10]
  init: ['k-means++']
  n_init: [10, 20]
  max_iter: [300]

Total combinations: 16


  [1/16] KMEANS_n_clusters3_initk-means++_n_init10_max_iter300: Sil=0.2544, CH=1269.22, DB=1.5338, Clusters=3
  [2/16] KMEANS_n_clusters3_initk-means++_n_init20_max_iter300: Sil=0.2544, CH=1269.22, DB=1.5338, Clusters=3
  [3/16] KMEANS_n_clusters4_initk-means++_n_init10_max_iter300: Sil=0.2355, CH=1030.48, DB=1.8631, Clusters=4
  [4/16] KMEANS_n_clusters4_initk-means++_n_init20_max_iter300: Sil=0.2355, CH=1030.48, DB=1.8631, Clusters=4
  [5/16] KMEANS_n_clusters5_initk-means++_n_init10_max_iter300: Sil=0.2352, CH=894.63, DB=1.7419, Clusters=5
  [6/16] KMEANS_n_clusters5_initk-means++_n_init20_max_iter300: Sil=0.2352, CH=894.63, DB=1.7419, Clusters=5
  [7/16] KMEANS_n_clusters6_initk-means++_n_init10_max_iter300: Sil=0.1937, CH=820.04, DB=1.8492, Clusters=6
  [8/16] KMEANS_n_clusters6_initk-means++_n_init20_max_iter300: Sil=0.1938, CH=820.06, DB=1.8499, Clusters=6
  [9/16] KMEANS_n_clusters7_initk-means++_n_init10_max_iter300: Sil=0.2019, CH=777.00, DB=1.6383, Clusters=7
  [10/16] KMEAN

model_type  n_clusters  silhouette  calinski_harabasz  davies_bouldin  composite_score
    kmeans           3    0.254433        1269.217570        1.533831         0.701773
    kmeans           3    0.254433        1269.217570        1.533831         0.701773
    kmeans           7    0.201851         776.998807        1.638289         0.354408

🏆 BEST K-MEANS CONFIG
  n_clusters             : 3
  init                   : k-means++
  n_init                 : 10

  Silhouette Score       : 0.2544
  Calinski-Harabasz      : 1269.22
  Davies-Bouldin Index   : 1.5338

🔄 Training best model...


💾 Labels saved: results/kmeans_best_composite_labels.csv


✅ Best K-Means model saved!
   Model: results/kmeans_composite_best.pkl
   Labels: results/kmeans_best_composite_labels.csv

📊 TOP 5 CONFIGS
 n_clusters      init  n_init  silhouette  calinski_harabasz  davies_bouldin
          3 k-means++      10    0.254433        1269.217570        1.533831
          3 k-means++      20    0.254433        1269.217570        1.533831
          7 k-means++      20    0.201851         776.998807        1.638289
          7 k-means++      10    0.201851         776.998807        1.638289
          5 k-means++      20    0.235202         894.634671        1.741909


In [ ]:
print("="*80)
print("DBSCAN HYPERPARAMETER TUNING - COMPOSITE METRIC")
print("="*80)

# 1. Thiết lập Grid cho DBSCAN
# eps: Khoảng cách tối đa giữa 2 điểm để được coi là cùng vùng lân cận
# min_samples: Số điểm tối thiểu để tạo thành 1 core point
# metric: Độ đo khoảng cách
dbscan_grid = {
    "eps": [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0],
    "min_samples": [5, 10, 15, 20],
    "metric": ["euclidean", "manhattan"]
}

print("Grid parameters:")
for key, values in dbscan_grid.items():
    print(f"  {key}: {values}")

total = 1
for v in dbscan_grid.values():
    total *= len(v)
print(f"\nTotal combinations: {total}")

# 2. Khởi tạo Tuner với COMPOSITE METRIC
evaluator = ClusteringEvaluator()
tuning_config = TuningConfig(
    data_path="data/processed/encoded_data.csv",
    results_path="results/dbscan_composite_tuning.csv",
    metric_selection="composite",  # Sử dụng composite score
    silhouette_weight=0.4,
    calinski_weight=0.3,
    davies_weight=0.3
)
tuner = HyperparameterTuner(config=tuning_config, evaluator=evaluator)

print(f"\n📊 Using Composite Score:")
print(f"   Silhouette weight     : {tuning_config.silhouette_weight}")
print(f"   Calinski-Harabasz wt  : {tuning_config.calinski_weight}")
print(f"   Davies-Bouldin wt     : {tuning_config.davies_weight}")

# 3. Chạy Tuning
print("\n🔄 Running DBSCAN grid search...")
print("⚠️  Note: DBSCAN có thể tạo nhiều noise points (cluster -1)")
tuner.run_grid_search("dbscan", dbscan_grid)

# 4. Lưu kết quả
tuner.save_results()

# 5. Lấy kết quả tốt nhất
df_results = tuner.get_summary()

if len(df_results) > 0:
    best = df_results.iloc[0]

    print("\n" + "="*80)
    print("🏆 BEST DBSCAN CONFIG (by Composite Score)")
    print("="*80)
    print(f"  eps                    : {best['eps']}")
    print(f"  min_samples            : {int(best['min_samples'])}")
    print(f"  metric                 : {best['metric']}")
    print(f"\n  📈 METRICS:")
    print(f"  Silhouette Score       : {best['silhouette']:.4f}")
    print(f"  Calinski-Harabasz      : {best['calinski_harabasz']:.2f}")
    print(f"  Davies-Bouldin Index   : {best['davies_bouldin']:.4f}")
    print(f"  Composite Score        : {best['composite_score']:.4f}")
    print(f"  Number of Clusters     : {int(best['n_clusters'])}")

    # Train và lưu model tốt nhất
    print("\n🔄 Training best model...")
    best_params = {
        "eps": float(best['eps']),
        "min_samples": int(best['min_samples']),
        "metric": best['metric']
    }

    config = TrainingConfig(
        data_path="data/processed/encoded_data.csv",
        model_type="dbscan",
        model_params=best_params,
        model_path="results/dbscan_composite_best.pkl"
    )

    trainer = ModelTrainer(config=config, evaluator=evaluator)
    trainer.load_data()
    trainer.train_model()
    
    # Kiểm tra số clusters và noise
    labels = trainer.get_cluster_labels()
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    noise_pct = n_noise / len(labels) * 100
    
    print(f"\n📊 Cluster Analysis:")
    print(f"  Total clusters (excluding noise): {n_clusters}")
    print(f"  Noise points: {n_noise} ({noise_pct:.1f}%)")
    
    trainer.save_model()
    trainer.save_labels("results/dbscan_composite_best_labels.csv")

    print("\n✅ Best DBSCAN model saved!")
    print("   Model: results/dbscan_composite_best.pkl")
    print("   Labels: results/dbscan_composite_best_labels.csv")


## 5D. DBSCAN Tuning

Tìm best configuration cho DBSCAN (Density-Based Spatial Clustering)

In [ ]:
print("="*80)
print("GMM (Gaussian Mixture Model) - COMPOSITE METRIC")
print("="*80)

# 1. Thiết lập Grid cho GMM
# n_clusters: Số lượng Gaussian components
# covariance_type: Loại ma trận covariance
#   - 'full': Mỗi component có ma trận covariance riêng (linh hoạt nhất)
#   - 'tied': Tất cả components dùng chung 1 ma trận covariance
#   - 'diag': Ma trận đường chéo (assume features độc lập)
#   - 'spherical': Ma trận đường chéo với variance bằng nhau
# n_init: Số lần khởi tạo ngẫu nhiên
# max_iter: Số vòng lặp tối đa cho EM algorithm
gmm_grid = {
    "n_clusters": [3, 4, 5, 6, 7, 8],
    "covariance_type": ["full", "tied", "diag", "spherical"],
    "n_init": [10],
    "max_iter": [200]
}

print("Grid parameters:")
for key, values in gmm_grid.items():
    print(f"  {key}: {values}")

total = 1
for v in gmm_grid.values():
    total *= len(v)
print(f"\nTotal combinations: {total}")

# 2. Khởi tạo Tuner với COMPOSITE METRIC
evaluator = ClusteringEvaluator()
tuning_config = TuningConfig(
    data_path="data/processed/encoded_data.csv",
    results_path="results/gmm_composite_tuning.csv",
    metric_selection="composite",
    silhouette_weight=0.4,
    calinski_weight=0.3,
    davies_weight=0.3
)
tuner = HyperparameterTuner(config=tuning_config, evaluator=evaluator)

print(f"\n📊 Using Composite Score:")
print(f"   Silhouette weight     : {tuning_config.silhouette_weight}")
print(f"   Calinski-Harabasz wt  : {tuning_config.calinski_weight}")
print(f"   Davies-Bouldin wt     : {tuning_config.davies_weight}")

# 3. Chạy Tuning
print("\n🔄 Running GMM grid search...")
print("💡 GMM là probabilistic model, mỗi điểm có xác suất thuộc từng cluster")
tuner.run_grid_search("gmm", gmm_grid)

# 4. Lưu kết quả
tuner.save_results()

# 5. Lấy kết quả tốt nhất
df_results = tuner.get_summary()

if len(df_results) > 0:
    best = df_results.iloc[0]

    print("\n" + "="*80)
    print("🏆 BEST GMM CONFIG (by Composite Score)")
    print("="*80)
    print(f"  n_clusters             : {int(best['n_clusters'])}")
    print(f"  covariance_type        : {best['covariance_type']}")
    print(f"  n_init                 : {int(best['n_init'])}")
    print(f"  max_iter               : {int(best['max_iter'])}")
    print(f"\n  📈 METRICS:")
    print(f"  Silhouette Score       : {best['silhouette']:.4f}")
    print(f"  Calinski-Harabasz      : {best['calinski_harabasz']:.2f}")
    print(f"  Davies-Bouldin Index   : {best['davies_bouldin']:.4f}")
    print(f"  Composite Score        : {best['composite_score']:.4f}")

    # Train và lưu model tốt nhất
    print("\n🔄 Training best model...")
    best_params = {
        "covariance_type": best['covariance_type'],
        "n_init": int(best['n_init']),
        "max_iter": int(best['max_iter'])
    }

    config = TrainingConfig(
        data_path="data/processed/encoded_data.csv",
        model_type="gmm",
        n_clusters=int(best['n_clusters']),
        model_params=best_params,
        model_path="results/gmm_composite_best.pkl"
    )

    trainer = ModelTrainer(config=config, evaluator=evaluator)
    trainer.load_data()
    trainer.train_model()
    trainer.save_model()
    trainer.save_labels("results/gmm_composite_best_labels.csv")

    print("\n✅ Best GMM model saved!")
    print("   Model: results/gmm_composite_best.pkl")
    print("   Labels: results/gmm_composite_best_labels.csv")

    # Show top 5 configs
    print("\n" + "="*80)
    print("📊 TOP 5 CONFIGS (by Composite Score)")
    print("="*80)
    cols = ['n_clusters', 'covariance_type', 'n_init',
            'silhouette', 'calinski_harabasz', 'davies_bouldin', 'composite_score']
    cols = [c for c in cols if c in df_results.columns]
    print(df_results[cols].head(5).to_string(index=False))
    
    # Phân tích covariance type performance
    print("\n" + "="*80)
    print("📊 COVARIANCE TYPE COMPARISON")
    print("="*80)
    if 'covariance_type' in df_results.columns:
        cov_analysis = df_results.groupby('covariance_type').agg({
            'silhouette': 'mean',
            'calinski_harabasz': 'mean',
            'davies_bouldin': 'mean',
            'composite_score': 'mean'
        }).round(4)
        print(cov_analysis)
        print("\n💡 Interpretation:")
        print("   - 'full': Tốt nhất nhưng tốn bộ nhớ (n_features²)")
        print("   - 'tied': Cân bằng giữa flexibility và efficiency")
        print("   - 'diag': Nhanh, phù hợp với high-dimensional data")
        print("   - 'spherical': Nhanh nhất nhưng ít linh hoạt")

else:
    print("⚠️ No valid results found")

In [ ]:
print("="*80)
print("HDBSCAN - COMPOSITE METRIC")
print("="*80)

# 1. Thiết lập Grid cho HDBSCAN
# min_cluster_size: Số điểm tối thiểu trong 1 cluster (quan trọng nhất!)
# min_samples: Số điểm trong neighborhood để tính core distance
# metric: Độ đo khoảng cách
# cluster_selection_method:
#   - 'eom' (Excess of Mass): Chọn cluster stable nhất
#   - 'leaf': Chọn leaf clusters trong hierarchy tree
hdbscan_grid = {
    "min_cluster_size": [10, 20, 30, 50, 100],
    "min_samples": [10, 20],
    "metric": ["euclidean", "manhattan"],
    "cluster_selection_method": ["eom"]
}

print("Grid parameters:")
for key, values in hdbscan_grid.items():
    print(f"  {key}: {values}")

total = 1
for v in hdbscan_grid.values():
    total *= len(v)
print(f"\nTotal combinations: {total}")

# 2. Khởi tạo Tuner với COMPOSITE METRIC
evaluator = ClusteringEvaluator()
tuning_config = TuningConfig(
    data_path="data/processed/encoded_data.csv",
    results_path="results/hdbscan_composite_tuning.csv",
    metric_selection="composite",
    silhouette_weight=0.4,
    calinski_weight=0.3,
    davies_weight=0.3
)
tuner = HyperparameterTuner(config=tuning_config, evaluator=evaluator)

print(f"\n📊 Using Composite Score:")
print(f"   Silhouette weight     : {tuning_config.silhouette_weight}")
print(f"   Calinski-Harabasz wt  : {tuning_config.calinski_weight}")
print(f"   Davies-Bouldin wt     : {tuning_config.davies_weight}")

# 3. Chạy Tuning
print("\n🔄 Running HDBSCAN grid search...")
print("💡 HDBSCAN advantages:")
print("   ✅ Tự động tìm số clusters (không cần chỉ định K)")
print("   ✅ Phát hiện clusters với mật độ khác nhau")
print("   ✅ Robust với noise và outliers")
tuner.run_grid_search("hdbscan", hdbscan_grid)

# 4. Lưu kết quả
tuner.save_results()

# 5. Lấy kết quả tốt nhất
df_results = tuner.get_summary()

if len(df_results) > 0:
    best = df_results.iloc[0]

    print("\n" + "="*80)
    print("🏆 BEST HDBSCAN CONFIG (by Composite Score)")
    print("="*80)
    print(f"  min_cluster_size          : {int(best['min_cluster_size'])}")
    print(f"  min_samples               : {int(best['min_samples'])}")
    print(f"  metric                    : {best['metric']}")
    print(f"  cluster_selection_method  : {best['cluster_selection_method']}")
    print(f"\n  📈 METRICS:")
    print(f"  Silhouette Score          : {best['silhouette']:.4f}")
    print(f"  Calinski-Harabasz         : {best['calinski_harabasz']:.2f}")
    print(f"  Davies-Bouldin Index      : {best['davies_bouldin']:.4f}")
    print(f"  Composite Score           : {best['composite_score']:.4f}")
    print(f"  Number of Clusters        : {int(best['n_clusters'])}")

    # Train và lưu model tốt nhất
    print("\n🔄 Training best model...")
    best_params = {
        "min_cluster_size": int(best['min_cluster_size']),
        "min_samples": int(best['min_samples']),
        "metric": best['metric'],
        "cluster_selection_method": best['cluster_selection_method']
    }

    config = TrainingConfig(
        data_path="data/processed/encoded_data.csv",
        model_type="hdbscan",
        model_params=best_params,
        model_path="results/hdbscan_composite_best.pkl"
    )

    trainer = ModelTrainer(config=config, evaluator=evaluator)
    trainer.load_data()
    trainer.train_model()
    
    # Kiểm tra số clusters và noise
    labels = trainer.get_cluster_labels()
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    noise_pct = n_noise / len(labels) * 100
    
    print(f"\n📊 Cluster Analysis:")
    print(f"  Total clusters (excluding noise): {n_clusters}")
    print(f"  Noise points: {n_noise} ({noise_pct:.1f}%)")
    
    # Phân bố kích thước clusters
    if n_clusters > 0:
        cluster_sizes = pd.Series(labels).value_counts().sort_index()
        cluster_sizes = cluster_sizes[cluster_sizes.index != -1]  # Loại noise
        print(f"\n  Cluster size distribution:")
        for cluster_id, size in cluster_sizes.items():
            pct = size / len(labels) * 100
            print(f"    Cluster {cluster_id}: {size} samples ({pct:.1f}%)")
    
    trainer.save_model()
    trainer.save_labels("results/hdbscan_composite_best_labels.csv")

    print("\n✅ Best HDBSCAN model saved!")
    print("   Model: results/hdbscan_composite_best.pkl")
    print("   Labels: results/hdbscan_composite_best_labels.csv")

    # Show top 5 configs
    print("\n" + "="*80)
    print("📊 TOP 5 CONFIGS (by Composite Score)")
    print("="*80)
    cols = ['min_cluster_size', 'min_samples', 'metric', 'n_clusters',
            'silhouette', 'calinski_harabasz', 'davies_bouldin', 'composite_score']
    cols = [c for c in cols if c in df_results.columns]
    print(df_results[cols].head(5).to_string(index=False))
    
    # Phân tích min_cluster_size impact
    print("\n" + "="*80)
    print("📊 MIN_CLUSTER_SIZE IMPACT ANALYSIS")
    print("="*80)
    if 'min_cluster_size' in df_results.columns:
        size_analysis = df_results.groupby('min_cluster_size').agg({
            'n_clusters': 'mean',
            'silhouette': 'mean',
            'calinski_harabasz': 'mean',
            'composite_score': 'mean'
        }).round(2)
        print(size_analysis)
        print("\n💡 Interpretation:")
        print("   - min_cluster_size nhỏ → nhiều clusters, có thể over-segment")
        print("   - min_cluster_size lớn → ít clusters, có thể under-segment")
        print("   - Chọn value cân bằng giữa số clusters và quality metrics")

else:
    print("⚠️ No valid results found")
    print("💡 Tip: HDBSCAN có thể không tìm được cấu hình tốt nếu:")
    print("   - min_cluster_size quá lớn so với dataset size")
    print("   - Data không có cấu trúc hierarchical density rõ ràng")

## 5F. HDBSCAN Tuning

Tìm best configuration cho HDBSCAN (Hierarchical Density-Based Spatial Clustering)

## 5E. GMM Tuning

Tìm best configuration cho Gaussian Mixture Model (GMM)